### pobranie danych źródłowych z portalu IMGW dla wybranego roku, miesiąca i zakresu danych

In [382]:
import requests, zipfile, io

month = 10
year = 2024

file = requests.get(f'https://danepubliczne.imgw.pl/datastore/getfiledown/Arch/Telemetria/Meteo/{year}/Meteo_{year}-{month}.zip')
# zip = zipfile.ZipFile(io.BytesIO(file.content))
# zip.extractall("dane_imgw")

## wczytanie danych meteo do redisa

In [383]:
import redis

"""Basic connection example.
"""

import redis

r = redis.Redis(
    host='redis-10759.c328.europe-west3-1.gce.redns.redis-cloud.com',
    port=10759,
    decode_responses=True,
    username="default",
    password="eVuJMCYwmfs79qqWNQDI9Zw7fcS23ugM",
)

success = r.set('foo', 'bar')
result = r.get('foo')
print(result)


bar


In [384]:
import pandas as pd

files = {
    "B00202A": "dane_imgw/B00202A_2024_10.csv",
    "B00300S": "dane_imgw/B00300S_2024_10.csv",
    "B00305A": "dane_imgw/B00305A_2024_10.csv",
    "B00604S": "dane_imgw/B00604S_2024_10.csv",
    "B00606S": "dane_imgw/B00606S_2024_10.csv",
    "B00608S": "dane_imgw/B00608S_2024_10.csv",
    "B00702A": "dane_imgw/B00702A_2024_10.csv",
    "B00703A": "dane_imgw/B00703A_2024_10.csv",
    "B00714A": "dane_imgw/B00714A_2024_10.csv",
    "B00802A": "dane_imgw/B00802A_2024_10.csv",
}

dataframes = {}
for key, path in files.items():
    df = pd.read_csv(path, header=None, sep=";")
    df.columns = ["kodSH", "parametrSH", "czas", f"{key}"]
    df['czas'] = pd.to_datetime(df['czas'], format="%Y-%m-%d %H:%M")
    dataframes[key] = df

base_key = "B00202A"
base_df = dataframes[base_key][["kodSH", "czas"]] .drop_duplicates() 

for key, df in dataframes.items():
    if key == base_key:
        continue 
    trimmed = df[["kodSH", "czas", f"{key}"]]
    base_df = pd.merge(base_df, trimmed, on=["kodSH", "czas"], how="left")

base_df['data'] = base_df['czas'].dt.date
base_df['time'] = base_df['czas'].dt.time
base_df = base_df.drop(columns=["czas"])
base_df

,kodSH,B00300S,B00305A,B00604S,B00606S,B00608S,B00702A,B00703A,B00714A,B00802A,data,time
0,249190090,3.29,0.17,NaN,0.0,0.0,0.0,0.9,NaN,92.06,2024-10-01,00:00:00
1,249190090,3.16,0.11,NaN,NaN,0.0,0.0,0.2,NaN,91.97,2024-10-01,00:10:00
2,249190090,3.05,0.03,NaN,NaN,0.0,0.0,0.7,NaN,91.98,2024-10-01,00:20:00
3,249190090,3.01,-0.02,NaN,NaN,0.0,0.2,1.4,NaN,92.45,2024-10-01,00:30:00
4,249190090,3.07,0.00,NaN,NaN,0.0,0.1,1.5,NaN,92.50,2024-10-01,00:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...
1166100,254150060,11.50,11.00,NaN,NaN,0.0,7.0,9.3,NaN,80.90,2024-10-30,18:10:00
1166101,254150060,11.40,10.60,NaN,NaN,0.0,6.7,9.2,NaN,80.90,2024-10-30,18:20:00
1166102,254150060,11.40,10.70,NaN,NaN,0.0,6.8,9.4,NaN,81.70,2024-10-30,18:30:00
1166103,254150060,11.40,10.90,NaN,NaN,0.0,6.2,8.7,NaN,82.00,2024-10-30,18:40:00


### wczytanie danych przestrzennych do struktur danych modułu GeoPandas

### koordynaty stacji

In [385]:
import geopandas as gpd

effacility = gpd.read_file("dane_przestrzenne/effacility.geojson")
effacility = effacility[["ifcid", "name1", "geometry"]]
effacility.rename(columns={"name1": "name"}, inplace=True)
effacility['x'] = float(effacility['geometry'].x)
effacility['y'] = float(effacility['geometry'].y)
effacility

TypeError: cannot convert the series to <class 'float'>

### wybranie odpowiednich danych z tabeli powiaty

In [ ]:
powiaty = powiaty = gpd.read_file("dane_przestrzenne/powiaty.shp")
powiaty = powiaty[["id", "name", "national_c", "geometry"]]
powiaty['woj_code'] = powiaty.apply(lambda x: x['national_c'][:2], axis=1)
powiaty.head()

,id,name,national_c,geometry,woj_code
0,5,złotoryjski,0226,"POLYGON ((269083.616 370884.075, 269164.192 37...",02
1,10,jeleniogórski,0206,"POLYGON ((244044.052 332419.671, 244053.906 33...",02
2,15,lubański,0210,"POLYGON ((228421.669 366620.161, 228453.763 36...",02
3,28,oleśnicki,0214,"POLYGON ((377122.144 378970.426, 377361.467 37...",02
4,32,bolesławiecki,0201,"POLYGON ((235157.110 379452.665, 235210.213 37...",02


### wybranie odpowiednich danych z tabeli województwa

In [ ]:
wojewodztwa = gpd.read_file("dane_przestrzenne/woj.shp")
wojewodztwa = wojewodztwa[["id", "name", "national_c", "geometry"]]
wojewodztwa.head()

,id,name,national_c,geometry
0,186,dolnośląskie,02,"POLYGON ((205901.102 343181.955, 205962.944 34..."
1,325,kujawsko-pomorskie,04,"POLYGON ((383436.881 613642.850, 383483.577 61..."
2,523,lubelskie,06,"POLYGON ((681015.792 420027.270, 681441.239 42..."
3,637,lubuskie,08,"POLYGON ((196217.270 448607.281, 196232.688 44..."
4,811,łódzkie,10,"POLYGON ((435569.180 387437.039, 435633.348 38..."


### połączenie powiatów z województwami

In [ ]:
merged = pd.merge(powiaty, wojewodztwa, left_on="woj_code", right_on="national_c", how="left")
merged = merged.drop(columns=["id_y", "national_c_y", "national_c_x", "woj_code", "id_x", "geometry_y"])
merged.rename(columns={"name_x": "powiat", "name_y": "wojewodztwo"}, inplace=True)
merged.set_geometry("geometry_x", inplace=True)

merged = gpd.sjoin(effacility, merged, how="left", op="within")
merged = merged.drop(columns=["index_right"])
merged

C:\Users\adria\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\adria\AppData\Local\Temp\ipykernel_31260\3268133761.py:6: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:2180
Right CRS: PROJCS["ETRS89_Poland_CS92",GEOGCS["ETRS89",DATUM[ ...

  merged = gpd.sjoin(effacility, merged, how="left", op="within")


,ifcid,name,geometry,x,y,powiat,wojewodztwo
0,149180010,Krzyżanowice,POINT (448926.076 236501.650),448926.076078,236501.650138,raciborski,śląskie
1,149180020,Chałupki,POINT (451760.530 228509.024),451760.530367,228509.024205,raciborski,śląskie
2,149180030,Łaziska,POINT (460035.038 228718.997),460035.038114,228718.996994,wodzisławski,śląskie
3,149180040,Gołkowice,POINT (463863.107 228846.279),463863.106694,228846.279355,wodzisławski,śląskie
4,149180050,Zebrzydowice,POINT (472228.166 223702.104),472228.165856,223702.103621,cieszyński,śląskie
...,...,...,...,...,...,...,...
2647,453220010,Jezioro Rajgrodzkie,POINT (741832.789 659022.551),741832.789480,659022.551063,grajewski,podlaskie
2648,454170010,Jezioro Jasień,POINT (409980.914 716533.986),409980.913648,716533.986151,bytowski,pomorskie
2649,454170020,Jezioro Raduńskie Górne,POINT (432806.035 707830.127),432806.035468,707830.127291,kartuski,pomorskie
2650,454170030,Jezioro Łebsko,POINT (402818.172 765146.628),402818.172307,765146.628034,słupski,pomorskie


### połączenie framów

In [ ]:
superbaza = pd.merge(base_df, merged, left_on="kodSH", right_on="ifcid", how="inner")
superbaza = superbaza.drop(columns=["ifcid"])
superbaza

,kodSH,B00300S,B00305A,B00604S,B00606S,B00608S,B00702A,B00703A,B00714A,B00802A,data,time,name,geometry,x,y,powiat,wojewodztwo
0,249190090,3.29,0.17,NaN,0.0,0.0,0.0,0.9,NaN,92.06,2024-10-01,00:00:00,Inwałd,POINT (527918.825 222282.573),527918.824642,222282.572526,wadowicki,małopolskie
1,249190090,3.16,0.11,NaN,NaN,0.0,0.0,0.2,NaN,91.97,2024-10-01,00:10:00,Inwałd,POINT (527918.825 222282.573),527918.824642,222282.572526,wadowicki,małopolskie
2,249190090,3.05,0.03,NaN,NaN,0.0,0.0,0.7,NaN,91.98,2024-10-01,00:20:00,Inwałd,POINT (527918.825 222282.573),527918.824642,222282.572526,wadowicki,małopolskie
3,249190090,3.01,-0.02,NaN,NaN,0.0,0.2,1.4,NaN,92.45,2024-10-01,00:30:00,Inwałd,POINT (527918.825 222282.573),527918.824642,222282.572526,wadowicki,małopolskie
4,249190090,3.07,0.00,NaN,NaN,0.0,0.1,1.5,NaN,92.50,2024-10-01,00:40:00,Inwałd,POINT (527918.825 222282.573),527918.824642,222282.572526,wadowicki,małopolskie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090433,254170210,12.10,12.90,NaN,NaN,0.1,2.4,3.1,NaN,97.20,2024-10-29,11:20:00,Borucino,POINT (434327.418 710403.930),434327.417613,710403.930419,kartuski,pomorskie
1090434,254170210,12.10,12.80,NaN,NaN,0.1,2.7,3.8,NaN,97.20,2024-10-29,11:30:00,Borucino,POINT (434327.418 710403.930),434327.417613,710403.930419,kartuski,pomorskie
1090435,254170210,12.10,12.80,NaN,NaN,0.0,2.6,3.2,NaN,97.40,2024-10-29,11:40:00,Borucino,POINT (434327.418 710403.930),434327.417613,710403.930419,kartuski,pomorskie
1090436,254170210,12.10,12.80,NaN,NaN,0.0,2.2,3.0,NaN,97.40,2024-10-29,11:50:00,Borucino,POINT (434327.418 710403.930),434327.417613,710403.930419,kartuski,pomorskie
